In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis as fva
from cobra.flux_analysis import pfba
sys.path.insert(0, r"C:\Users\Bisbii\PythonProjects\ExpAlgae\src")
from ExpAlgae.model.COBRAmodel import *
import seaborn as sns
sns.set(rc={'figure.figsize':(35,8.27)})

In [2]:
def read_model(data_directory):
    model_to_load = MyModel(join(data_directory, "models/model_with_trials.xml"), "e_Biomass__cytop")
    model_to_load.add_medium(join(data_directory, "media.xlsx"), "base_medium")
    model_to_load.exchanges.EX_C00009__dra.bounds = (-0.05, 1000)
    model_to_load.exchanges.EX_C00244__dra.bounds = (-5, 1000)
    return model_to_load

model = read_model("../data")

Loading

Reactions: 4602
Metabolites: 3691
Genes: 1684
Model loaded


In [7]:
consistent_model = cobra.flux_analysis.fastcc(model)

In [ ]:
from cobra.flux_analysis import add_loopless

with model:
    add_loopless(model)
    try:
        solution = model.optimize()
    except:
        print('model is infeasible')

In [11]:
from cobra.flux_analysis import find_blocked_reactions

with model:
    blocked = find_blocked_reactions(model)
    print(len(model.reactions) - len(blocked))

3546


In [9]:
len(consistent_model.reactions)

3541

In [13]:
with consistent_model:
    blocked = find_blocked_reactions(consistent_model)
    print(len(consistent_model.reactions) - len(blocked))

3541


In [3]:
print(model.test_reaction("BMGR1612__chlo"))

None
None
None
None
                                                    Flux
Monogalactosyldiacylglycerol (1-(9Z,12Z,15Z)-oc...     0
Reduced ferredoxin                                     0
Oxygen                                                 0
H+                                                     0


In [30]:
model.metabolites.BMGC144835__lip.summary(fva=0.0)

Percent,Flux,Range,Reaction,Definition
100.00%,2.025E-05,[0; 0.0009697],BMGR4068__ermem,BMGC144835__er <=> BMGC144835__lip
Percent,Flux,Range,Reaction,Definition
0.00%,0,[-2.81E-05; 0],BMGR5845__lip,BMGC144835__lip + C00001__cytop --> BMGC70110__lip + C00080__cytop + C06425__cytop
100.00%,-2.025E-05,[-0.0009697; 0],e_TAG__lip,0.0023 BMGC102109__lip + 0.0046 BMGC105358__lip + 0.0618 BMGC107726__lip + 0.0044 BMGC111414__lip + 0.0088 BMGC111564__lip + 0.01017 BMGC113284__lip + 0.0023 BMGC116699__lip + 0.0069 BMGC144835__lip + 0.0251 BMGC154891__lip + 0.0167 BMGC155553__lip + 0.006 BMGC161268__lip + 0.0029 BMGC172238__lip + 0.0379 BMGC177299__lip + 0.0118 BMGC186981__lip + 0.0415 BMGC215189__lip + 0.247 BMGC223029__lip + 0.0172 BMGC229670__lip + 0.0514 BMGC55333__lip + 0.0339 BMGC57008__lip + 0.0681 BMGC66372__lip + 0.09737 BMGC74677__lip + 0.0406 BMGC77019__lip + 0.0068 BMGC80156__lip + 0.0091 BMGC81936__lip + 0.0089 BMGC86896__lip + 0.02917 BMGC91317__lip + 0.0022 BMGC97019__lip + 0.0045 BMGC98270__lip + 0.0047 BMGC99062__lip + 0.05145 tag1819Z164Z7Z12Z15Z1819Z__lip + 0.0585 tag1829Z12Z164Z7Z12Z15Z1829Z12Z__lip + 0.02595 tag1839Z12Z15Z164Z7Z12Z15Z1839Z12Z15Z__lip --> C00422__lip


In [64]:
model.metabolites.BMGC41539__chlo.name

'1-dodecanoyl-2-(9Z-octadecenoyl)-sn-glycero-3-phosphate'

In [5]:
with model:
    # model.create_sink("mgdg140160__chlo")
    # model.create_demand("BMGC92261__lip")
    print(fva(model, reaction_list = model.reactions.DGDGS_183_164__chlo, fraction_of_optimum=0.1))

                     minimum   maximum
DGDGS_183_164__chlo      0.0  0.000165


In [3]:
pfba_solution = pfba(model)

In [4]:
df = pfba_solution.to_frame()
shadow_prices = pfba_solution.shadow_prices
shadow_prices = shadow_prices.loc[shadow_prices.index.str.contains("__extr")]

In [5]:
reduced_costs_greater_zero = df.loc[(df.index.str.contains("EX_")) | (df.index.str.contains("DM_"))].loc[df.reduced_costs > 0]

In [6]:
with model:
    for reaction in reduced_costs_greater_zero.index:
        copy_model = model.copy()
        met = model.metabolites.get_by_id(reaction.replace("EX_", "").replace("DM_", "").split("__")[0] + "__" + reaction.split("__")[1].replace("dra", "extr"))
        print(met.id, met.name)
        print(round(copy_model.optimize().objective_value, 3))
        copy_model.reactions.get_by_id(reaction).lower_bound = -1000
        print(round(copy_model.optimize().objective_value, 3))
        print("#"*200)

C00526__extr Deoxyuridine
0.203
0.204
########################################################################################################################################################################################################
C02823__extr Cyanocobalamin
0.203
0.203
########################################################################################################################################################################################################
C00267__extr alpha-D-Glucose
0.203
0.203
########################################################################################################################################################################################################
C01694__extr Ergosterol
0.203
0.203
########################################################################################################################################################################################################
C00106__extr Uracil
0.203
0.317
#########

In [7]:
with model:
    model.exchanges.EX_C00086__dra.lower_bound = -1000
    print(model.metabolites.C00086__cytop.summary())

C00086__cytop
Formula: CH4N2O

Producing Reactions
-------------------
Percent  Flux       Reaction                                                     Definition
100.00% 4.254  T_UREAt__plas  C00080__extr + C00086__extr --> C00080__cytop + C00086__cytop

Consuming Reactions
-------------------
Percent   Flux       Reaction                                                                      Definition
100.00% -4.254  R00774__cytop  C00002__cytop + C00086__cytop + C00288__cytop --> C00008__cytop + C00009__c...


In [15]:
with model:
    growth_range = np.arange(0, 0.8, 0.1)
    for ex in model.exchanges:
        if ex.lower_bound < 0:
            ex.lower_bound = -10000
    for reaction in model.reactions:
        if reaction.lower_bound < -1000:
            reaction.lower_bound = -10000
        if reaction.upper_bound < 1000:
            reaction.upper_bound = 10000
    model.minimize_uptake_sum()
    for growth in growth_range:
        model.reactions.e_Biomass__cytop.bounds = (growth, growth)
        sol = pfba(model)
        print(sol['e_Biomass__cytop'])
        print(sol['EX_C00009__dra'])
        print(sol['EX_C00244__dra'])
        print(sol['EX_C00009__dra']/sol['EX_C00244__dra'])
        print("#"*100)

0.0
0.0
0.0
nan
####################################################################################################
0.1
-0.015762919365495898
-0.25897466249341977
0.0608666470060422
####################################################################################################
0.2
-0.03152583873102205
-0.51794932498683
0.06086664700610173
####################################################################################################
0.30000000000000004
-0.04728875809736088
-0.7769239874824028
0.06086664700699818
####################################################################################################
0.4
-0.06305167746325768
-1.0358986499767397
0.0608666470070923
####################################################################################################
0.5
-0.07881459682790926
-1.2948733124676077
0.06086664700635018
####################################################################################################
0.6000000000000001
-0.09457751619290278

In [20]:
0.30/7.5

0.04